In [ ]:
!pip3 install pdfplumber
!pip install opencc-python-reimplemented
!pip install tabula-py
!pip install PyPDF2
!pip install fitz
!pip install pytesseract
!pip install frontend

In [48]:
import requests
import pdfplumber
from io import BytesIO
import csv
import pandas as pd
from opencc import OpenCC
import numpy as np
import ast
import jieba
from datetime import datetime
import re
from dateutil.relativedelta import relativedelta
from tabula import read_pdf
import PyPDF2
import pytesseract
from PIL import Image
import io

In [ ]:
read_pdf('A123456789.pdf', lattice=True, multiple_tables=True)

In [50]:

def extract_autobio(pdf_file_name, keyword, keyword2):
  with pdfplumber.open(pdf_file_name) as pdf:
      for page in pdf.pages:
          page_text = page.extract_text()
          if keyword in page_text:
              index_keyword = page_text.find(keyword)
              index_keyword2 = page_text.find(keyword2, index_keyword)
              if index_keyword2 != -1:
                  extracted_text = page_text[index_keyword2 + len(keyword2):]
                  # Find the last period in the extracted text
                  last_period_index = extracted_text.rfind("。")
                  if last_period_index != -1:
                      extracted_text = extracted_text[:last_period_index+1]  # Include the last period
                  return extracted_text.strip()  # Remove leading/trailing whitespace

def extract_table(pdf_file_name):
  # Try to read tables from the PDF
  dfs = read_pdf(pdf_file_name, lattice=True, multiple_tables=True)
  concatenated_df = pd.concat(dfs, ignore_index=True)
  return concatenated_df


keyword = "自傳內容"
keyword2 = "---------------------------------------------------------------------"

In [ ]:
extract_autobio('A123456789.pdf', keyword, keyword2)

In [ ]:
extract_autobio('B123456789.pdf', keyword, keyword2)

In [ ]:
extract_table('A123456789.pdf')

In [ ]:
extract_table('B123456789.pdf')

In [55]:
def clean_items(items):
    clean_items = [item.replace(' ', '').replace('\r', '').replace('\n', '').replace('(最高)', '').replace('(次高)', '') for item in items if isinstance(item, str) and '專業證照' not in item]
    clean_items = [x if x != "" else None for x in clean_items]
    return clean_items

def extract_and_clean_data(data, resume_row, keyword_1, keyword_2, length):
    data_section = []
    while keyword_1 not in str(resume_row):
        resume_row = next(resume_tuples)
    resume_row = next(resume_tuples)

    while keyword_2 not in str(resume_row):
        data_section.append(resume_row)
        resume_row = next(resume_tuples)

    for s in data_section:
    # Check if s[0] is a number and not NaN
        if isinstance(s[1], str):
            data.extend(clean_items(s))

    data.extend([''] * (length - len(data)))
    return data, resume_row

In [ ]:
############################ 撈出資料 ######################
all_data = []

for name in ['A123456789.pdf', 'B123456789.pdf']:
  resume_df = extract_table(name)
  resume_tuples = resume_df.itertuples(index=False, name=None)

  data = [name]
  licience = []
  try:
      resume_row = None
      data, resume_row = extract_and_clean_data(data, resume_row, '學歷背景', '工作經驗', 10)
      data, resume_row = extract_and_clean_data(data, resume_row, '工作經驗', '外語能力', 26)

      data.append(resume_row[1]) # 外語能力

      while '證照' not in str(resume_row):
          resume_row = next(resume_tuples)

      licience = []

      while True:
          licience.extend(resume_row)
          resume_row = next(resume_tuples)


  except StopIteration:
      pass
  data.append(clean_items(licience))
  data.append(extract_autobio(name, keyword, keyword2))
  all_data.append(data)

# Convert the list of lists to a DataFrame
result_df = pd.DataFrame(all_data)
result_df.to_csv('output.csv', index=False)